In [2]:
"""


9-29-18, version 4:
In last version, we figured out how to progressively split data, 
and how to make tf progressively read and train on large dataset.
# last rsme with 5 raw features w/ LineraR = 9.8

In this version, we want to add feature engineering onto it,
and hopefully improve the rsme largely.

Ok..met one very strange error as the fare_amount input at step 6701 with batch size = 128*3.
Not sure what was wrong about.. and what made it worse... it seems hard to print out the error line
in the fly of tf estimator to see what went wrong.

###################
9-14-18:
In v1 file, we built the model with few extra feature and used the simplest model (linear regression)

In last v2 file, we added more features (both bucket location, and several time features),
we also applied a DNN instead of linear regression to train the model.

In this v3 file, we want to improve the data input---we are not satisfied by just reading
the first million data. As a result, we want to read that large dataset progressively.
Reference Source = 
https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/03_tensorflow/c_dataset.ipynb



"""

'\n\n\n9-29-18, version 4:\nIn last version, we figured out how to progressively split data, \nand how to make tf progressively read and train on large dataset.\n# last rsme with 5 raw features w/ LineraR = 9.8\n\nIn this version, we want to add feature engineering onto it,\nand hopefully improve the rsme largely.\n\nOk..met one very strange error as the fare_amount input at step 6701 with batch size = 128*3.\nNot sure what was wrong about.. and what made it worse... it seems hard to print out the error line\nin the fly of tf estimator to see what went wrong.\n\n###################\n9-14-18:\nIn v1 file, we built the model with few extra feature and used the simplest model (linear regression)\n\nIn last v2 file, we added more features (both bucket location, and several time features),\nwe also applied a DNN instead of linear regression to train the model.\n\nIn this v3 file, we want to improve the data input---we are not satisfied by just reading\nthe first million data. As a result, w

In [ ]:
dirctory = "/Users/toby/Downloads/Data/Kaggle_Taxi/"
# tensorboard --logdir=/Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1
# localhost:6006

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import time
from math import sqrt
import random
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split

os.getcwd()


/Users/toby/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'/Users/toby/Documents/PycharmProjects/Kaggle/NYC_taxiFare_Prediction'

In [4]:
# data =  pd.read_csv(dirctory + 'chunk_eval_v2.csv', nrows = 10000, header = None)
# data.to_csv(dirctory + 'chunk_eval_v2_short.csv', index = False, mode = "w", header = False)
# # data =  pd.read_csv('nyc_taxi_train.csv', nrows = 1000)

# print(data.info())
# data.head()
# train_df, eval_df = train_test_split(data, test_size = 0.4)
# print(train_df.dtypes)
# train_df.head()



In [5]:
# data.columns

In [6]:
# all the supplying functions 
def late_night (row):
    if (row['hour'] <= 6) or (row['hour'] >= 20):
        return 1
    else:
        return 0


def night (row):
    if ((row['hour'] <= 20) and (row['hour'] >= 16)) and (row['weekday'] < 5):
        return 1
    else:
        return 0



def clean_df(df):
    df = df.dropna(how = 'any', axis = 'rows')
    df = df[df["fare_amount"] > 2.5]
    df = df[(df["passenger_count"] > 0) & (df["passenger_count"] <= 4)]
    df = df[(df["dropoff_longitude"] > -76) & (df["dropoff_longitude"] < -72)]
    df = df[(df["pickup_longitude"] > -76) & (df["pickup_longitude"] < -72)]
    df = df[(df["pickup_latitude"] > 38) & (df["pickup_latitude"] < 42)]
    df = df[(df["dropoff_latitude"] > 38) & (df["dropoff_latitude"] < 42)]  
    
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
    
    df['distance'] = ((df.abs_diff_longitude)**2 + \
                      (df.abs_diff_latitude)**2).apply(lambda x: 69.172*sqrt(x))

    df = df[((df["distance"] <= 25) & (df["fare_amount"] <= 50))\
            | ((df["distance"] > 25) & (df["fare_amount"] >= 50))]
    return df


def add_more_features(df):
    """here.."""
    df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
    df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
    df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
    df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
    df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
    df['night'] = df.apply (lambda x: night(x), axis=1)
    df['late_night'] = df.apply (lambda x: late_night(x), axis=1)    
    df = df.drop("pickup_datetime",1)
    return df

# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
    x = add_more_features(df),
    y = df['fare_amount'],
    batch_size = 128*3, # no sure why..
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

def create_feature_cols():
    return [
    tf.feature_column.numeric_column('passenger_count'),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()), 
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()),         
    tf.feature_column.numeric_column('abs_diff_longitude'),
    tf.feature_column.numeric_column('abs_diff_latitude'),
    tf.feature_column.numeric_column('distance'),
    tf.feature_column.numeric_column('year'),
    tf.feature_column.numeric_column('month'),
    tf.feature_column.numeric_column('day'),
    tf.feature_column.numeric_column('hour'),
    tf.feature_column.numeric_column('weekday'),
    tf.feature_column.numeric_column('night'),
    tf.feature_column.numeric_column('late_night')        
  ]


In [7]:
# ?tf.decode_csv

In [8]:
# CSV_COLUMNS = ['key','fare_amount', 'pickup_datetime', 'pickup_longitude',
#                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
#                'passenger_count']
# DEFAULTS = [['nokey'],[0.0], ["None"], [-74.0], [40.0], [-74.0], [40.7], [1.0], ]

CSV_COLUMNS = [
    'key', 'fare_amount', 'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
    'abs_diff_longitude', 'abs_diff_latitude', 'distance', 'year', 'month',
    'day', 'hour', 'weekday', 'night', 'late_night']

LABEL_COLUMN = 'fare_amount'
DEFAULTS = [['None'],[0.0], [-74.0], [40.0], [-74.0], [40.0], [1.0], 
            [0.0],[0.0],[0.0],[0],[0],[0],[0],[0],[0],[0]]

def read_dataset(filename, mode, batch_size = 512 * 2):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults = DEFAULTS) 
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label
        
        dataset = tf.data.TextLineDataset(filename).skip(1).map(decode_csv)

        # Note:
        # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
        # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)


        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # loop indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this


        dataset = dataset.repeat(num_epochs).batch(batch_size)

        return dataset.make_one_shot_iterator().get_next()
    return _input_fn



def get_train():
    return read_dataset(dirctory + 'chunk_train_v2.csv', mode = tf.estimator.ModeKeys.TRAIN)
#     return read_dataset(r'nyc_taxi_df_train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
    return read_dataset(dirctory + 'chunk_eval_v2_short.csv', mode = tf.estimator.ModeKeys.EVAL)
#     return read_dataset(r'nyc_taxi_df_eval.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
    return read_dataset(dirctory + 'featured_test.csv', mode = tf.estimator.ModeKeys.PREDICT)

In [9]:
# ?tf.estimator.LinearRegressor

In [10]:
# v3 old:
# INPUT_COLUMNS = [
#     tf.feature_column.numeric_column('pickup_longitude'),
#     tf.feature_column.numeric_column('pickup_latitude'),
#     tf.feature_column.numeric_column('dropoff_latitude'),
#     tf.feature_column.numeric_column('dropoff_longitude'),
#     tf.feature_column.numeric_column('passenger_count'),
# ]

# v4 new:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('passenger_count'),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()), 
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()),         
    tf.feature_column.numeric_column('abs_diff_longitude'),
    tf.feature_column.numeric_column('abs_diff_latitude'),
    tf.feature_column.numeric_column('distance'),
    tf.feature_column.numeric_column('year'),
    tf.feature_column.numeric_column('month'),
    tf.feature_column.numeric_column('day'),
    tf.feature_column.numeric_column('hour'),
    tf.feature_column.numeric_column('weekday'),
    tf.feature_column.numeric_column('night'),
    tf.feature_column.numeric_column('late_night')        
  ]

def add_more_features(feats):
    # Nothing to add (yet!)
    return feats

feature_cols = add_more_features(INPUT_COLUMNS)


In [11]:
# use this for first time training...

# tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = dirctory + 'taxi_local_v1'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
# model = tf.estimator.LinearRegressor(feature_columns = feature_cols, model_dir = OUTDIR)
model = tf.estimator.DNNRegressor(feature_columns = feature_cols, 
                                  model_dir = OUTDIR,
                                  hidden_units=[1024,512,512,256],
                                  optimizer=tf.train.ProximalAdagradOptimizer(
                                      learning_rate=0.01,
                                      l2_regularization_strength=0.02)
                                 )
model.train(input_fn = get_train(), steps = 8000) # use steps limit to tune the parameters
# model.train(input_fn = get_train(), max_steps=None)




In [12]:
# ? tf.estimator.LinearRegressor.train
# ? pd.DataFrame.to_csv

In [13]:
# use this for continuing training

# restore training 9-30-2018
# since the training didn't finish last time
# OUTDIR = dirctory + 'taxi_local_v1'
# model = tf.estimator.DNNRegressor(feature_columns = feature_cols, 
#                                   model_dir = OUTDIR,
#                                   hidden_units=[512,512,256],
#                                   optimizer=tf.train.ProximalAdagradOptimizer(
#                                       learning_rate=0.01,
#                                       l2_regularization_strength=0.02)
#                                  )
# model.train(input_fn = get_train(), steps = 8000) # use steps limit to tune the parameters
# model.train(input_fn = get_train(), max_steps=None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11e4cd080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /Use

INFO:tensorflow:global_step/sec: 14.8859
INFO:tensorflow:loss = 13445.65, step = 7301 (6.718 sec)
INFO:tensorflow:global_step/sec: 14.8978
INFO:tensorflow:loss = 10899.579, step = 7401 (6.712 sec)
INFO:tensorflow:global_step/sec: 14.8039
INFO:tensorflow:loss = 12071.794, step = 7501 (6.755 sec)
INFO:tensorflow:global_step/sec: 14.2834
INFO:tensorflow:loss = 14863.404, step = 7601 (7.001 sec)
INFO:tensorflow:global_step/sec: 14.795
INFO:tensorflow:loss = 10238.288, step = 7701 (6.759 sec)
INFO:tensorflow:global_step/sec: 14.7703
INFO:tensorflow:loss = 10674.867, step = 7801 (6.771 sec)
INFO:tensorflow:global_step/sec: 14.7256
INFO:tensorflow:loss = 14039.779, step = 7901 (6.790 sec)
INFO:tensorflow:Saving checkpoints for 8000 into /Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1/model.ckpt.
INFO:tensorflow:Loss for final step: 9682.946.


In [15]:
#save the model
feature_space = tf.feature_column.make_parse_example_spec(feature_cols)

serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_space)
model.export_savedmodel('export', serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1/model.ckpt-8000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/temp-b'1538729042'/saved_model.pb


b'export/1538729042'

In [16]:
def print_rmse(model, name, input_fn):
    metrics = model.evaluate(input_fn = input_fn, steps = None)
    print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'eval', get_valid()) #well...this evaluation is realllly taking long

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-05-08:44:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-05-08:44:03
INFO:tensorflow:Saving dict for global step 8000: average_loss = 12.850069, global_step = 8000, label/mean = 10.67352, loss = 12848.784, prediction/mean = 11.0478115
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: /Users/toby/Downloads/Data/Kaggle_Taxi/taxi_local_v1/model.ckpt-8000
RMSE on eval dataset = 3.5846993923187256


In [17]:
# notes:
# 1. [1024,512,512,256], 0.01, l1 w/ 0.02, train step = 5000 batch = 512 * 2, 4.2
# 2. [1024,512,256], 0.01, l1 w/ 0.02, train step = 8000 batch = 512 * 2, 3.55
# 3. [512,512,256], 0.01, l2 w/ 0.02, train step = 8000 batch = 512 * 2, 3.58


In [18]:
# ? model.predict

In [19]:
# def make_pred_input_fn(df, num_epochs):
#     return tf.estimator.inputs.pandas_input_fn(
#     x = df,
#     y = None,
#     batch_size = 128,
#     num_epochs = num_epochs,
#     shuffle = False,
#     queue_capacity = 1000,
#     num_threads = 1)

# test_df = pd.read_csv(dirctory + "featured_test.csv")
# test_df.head()

# # model stopped around step 720301 is saved to b'export/1538361964'

In [20]:
# result = model.predict(input_fn = make_pred_input_fn(test_df,1))
# result_df = pd.DataFrame(result)
# result_df['predictions'] = result_df['predictions'].apply(lambda x: float(x))
# result_df.head()

In [21]:
# result = model.predict(input_fn = make_pred_input_fn(test_df,1))
# result_df = pd.DataFrame(result)
# result_df['predictions'] = result_df['predictions'].apply(lambda x: float(x))
# result_df.head()

In [22]:
# sub = result_df.merge(test_df, left_index=True, right_index=True)[["predictions","key"]]
# sub.set_index("key", inplace = True)
# sub.columns = ["fare_amount"]
# sub.head()

In [23]:
# sub.to_csv(dirctory + "taxi_submission_full.csv", mode = 'w')

In [24]:
# result = model.predict(input_fn = get_test())
# result_df = pd.DataFrame(result)

# result_df.head()

In [25]:
# OUTDIR = 'taxi_trained'

In [26]:

os.system("Toby, 'job run finished!'")

32512

In [1]:
# os.getcwd()